##### Copyright 2024 Google LLC.

In [7]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Explore vision capabilities with the Gemini API

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs/vision"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on ai.google.dev</a>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/gemini-api/docs/vision.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/gemini-api/docs/vision.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

The Gemini API is able to process images and videos, enabling a multitude of
 exciting developer use cases. Some of Gemini's vision capabilities include
 the ability to:

*   Caption and answer questions about images
*   Transcribe and reason over PDFs, including long documents up to 2 million token context window
*   Describe, segment, and extract information from videos,
including both visual frames and audio, up to 90 minutes long
*   Detect objects in an image and return bounding box coordinates for them

This tutorial demonstrates some possible ways to prompt the Gemini API with
images and video input, provides code examples,
and outlines prompting best practices with multimodal vision capabilities.
All output is text-only.

## Setup

Before you use the File API, you need to install the Gemini API SDK package and configure an API key. This section describes how to complete these setup steps.

### Install the Python SDK and import packages

The Python SDK for the Gemini API is contained in the [google-generativeai](https://pypi.org/project/google-generativeai/) package. Install the dependency using pip.

In [8]:
!pip install -q -U google-generativeai

Import the necessary packages.

In [9]:
import google.generativeai as genai
from IPython.display import Markdown

### Set up your API key

The File API uses API keys for authentication and access. Uploaded files are associated with the project linked to the API key. Unlike other Gemini APIs that use API keys, your API key also grants access to data you've uploaded to the File API, so take extra care in keeping your API key secure. For more on keeping your keys
secure, see [Best practices for using API
keys](https://support.google.com/googleapi/answer/6310037).

Store your API key in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or are unfamiliar with Colab Secrets, refer to the [Authentication quickstart](https://github.com/google-gemini/gemini-api-cookbook/blob/main/quickstarts/Authentication.ipynb).

In [10]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

## Prompting with images

In this tutorial, you will upload images using the File API or as inline data and generate content based on those images.

### Technical details (images)
Gemini 1.5 Pro and Flash support a maximum of 3,600 image files.

Images must be in one of the following image data [MIME types](https://developers.google.com/drive/api/guides/ref-export-formats):

-   PNG - `image/png`
-   JPEG - `image/jpeg`
-   WEBP - `image/webp`
-   HEIC - `image/heic`
-   HEIF - `image/heif`

Each image is equivalent to 258 tokens.

While there are no specific limits to the number of pixels in an image besides the model’s context window, larger images are scaled down to a maximum resolution of 3072x3072 while preserving their original aspect ratio, while smaller images are scaled up to 768x768 pixels. There is no cost reduction for images at lower sizes, other than bandwidth, or performance improvement for images at higher resolution.

For best results:

*   Rotate images to the correct orientation before uploading.
*   Avoid blurry images.
*   If using a single image, place the text prompt after the image.

## Image input

For total image payload size less than 20MB, it's recommended to either upload
base64 encoded images or directly upload locally stored image files.

### Base64 encoded images

You can upload public image URLs by encoding them as Base64 payloads.
You can use the httpx library to fetch the image URLs.
The following code example shows how to do this:

In [11]:
import httpx
import base64

# Retrieve an image
image_path = "https://upload.wikimedia.org/wikipedia/commons/thumb/8/87/Palace_of_Westminster_from_the_dome_on_Methodist_Central_Hall.jpg/2560px-Palace_of_Westminster_from_the_dome_on_Methodist_Central_Hall.jpg"
image = httpx.get(image_path)

# Choose a Gemini model
model = genai.GenerativeModel(model_name="gemini-1.5-pro")

# Create a prompt
prompt = "Caption this image."
response = model.generate_content(
    [
        {
            "mime_type": "image/jpeg",
            "data": base64.b64encode(image.content).decode("utf-8"),
        },
        prompt,
    ]
)

Markdown(">" + response.text)

>A view of London, England featuring the Palace of Westminster (Houses of Parliament) and Big Ben, the London Eye, and a glimpse of the Shard in the background. The foreground showcases other buildings and rooftops. A dramatic, cloudy sky fills the frame.

### Multiple images

To prompt with multiple images in Base64 encoded format, you can do the
following:

In [12]:
import httpx
import base64

# Retrieve two images
image_path_1 = "https://upload.wikimedia.org/wikipedia/commons/thumb/8/87/Palace_of_Westminster_from_the_dome_on_Methodist_Central_Hall.jpg/2560px-Palace_of_Westminster_from_the_dome_on_Methodist_Central_Hall.jpg"
image_path_2 = "https://storage.googleapis.com/generativeai-downloads/images/jetpack.jpg"

image_1 = httpx.get(image_path_1)
image_2 = httpx.get(image_path_2)

# Create a prompt
prompt = "Generate a list of all the objects contained in both images."

response = model.generate_content([
{'mime_type':'image/jpeg', 'data': base64.b64encode(image_1.content).decode('utf-8')},
{'mime_type':'image/jpeg', 'data': base64.b64encode(image_2.content).decode('utf-8')}, prompt])

Markdown(response.text)

**Image 1 (London Skyline):**

* Houses of Parliament (including Big Ben/Elizabeth Tower)
* London Eye
* The Shard
* River Thames
* Various other buildings and skyscrapers
* Trees
* Construction cranes
* Cloudy sky

**Image 2 (Jetpack Backpack Sketch):**

* Backpack shape
* Padded strap support
* USB-C charging port
* Retractible boosters
* Steam exhaust clouds
* Text labels indicating:
    * "Jetpack Backpack"
    * "Fits 18" Laptop"
    * "Lightweight"
    * "Looks like a normal backpack"
    * "Retractible boosters"
    * "Steam-powered, Green/Clean"
    * "Padded strap support"
    * "USB-C charging"
    * "15-min battery life"

### Upload one or more locally stored image files

Alternatively, you can upload one or more locally stored image files..

You can download and use our drawings of [piranha-infested waters](https://storage.googleapis.com/generativeai-downloads/images/piranha.jpg) and a [firefighter with a cat](https://storage.googleapis.com/generativeai-downloads/images/firefighter.jpg). First, save these files to your local directory.

Then click **Files** on the left sidebar. For each file, click the **Upload** button, then navigate to that file's location and upload it:

<img width=400 src="https://ai.google.dev/tutorials/images/colab_upload.png">

When the combination of files and system instructions that you intend to send is larger than 20 MB in size, use the File API to upload those files. Smaller files can instead be called locally from the Gemini API:


In [13]:
import PIL.Image

sample_file_2 = PIL.Image.open('piranha.jpg')
sample_file_3 = PIL.Image.open('firefighter.jpg')

FileNotFoundError: [Errno 2] No such file or directory: '/content/piranha.jpg'

In [14]:
import google.generativeai as genai

# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Create a prompt.
prompt = "Write an advertising jingle based on the items in both images."

response = model.generate_content([sample_file_2, sample_file_3, prompt])

Markdown(response.text)

NameError: name 'sample_file_2' is not defined

Note that these inline data calls don't include many of the features available
through the File API, such as getting file metadata,
[listing](https://ai.google.dev/gemini-api/docs/vision?lang=python#list-files),
or [deleting files](https://ai.google.dev/gemini-api/docs/vision?lang=python#delete-files).

### Large image payloads

#### Upload an image file using the File API

When the combination of files and system instructions that you intend to send is larger than 20 MB in size, use the File API to upload those files.

**NOTE**: The File API lets you store up to 20 GB of files per project, with a per-file maximum size of 2 GB. Files are stored for 48 hours. They can be accessed in that period with your API key, but cannot be downloaded from the API. It is available at no cost in all regions where the Gemini API is available.

Upload the image using [`media.upload`](https://ai.google.dev/api/rest/v1beta/media/upload) and print the URI, which is used as a reference in Gemini API calls.

In [15]:
!curl -o jetpack.jpg https://storage.googleapis.com/generativeai-downloads/images/jetpack.jpg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  349k  100  349k    0     0  2374k      0 --:--:-- --:--:-- --:--:-- 2391k


In [16]:
# Upload the file and print a confirmation.
sample_file = genai.upload_file(path="jetpack.jpg",
                            display_name="Jetpack drawing")

print(f"Uploaded file '{sample_file.display_name}' as: {sample_file.uri}")

Uploaded file 'Jetpack drawing' as: https://generativelanguage.googleapis.com/v1beta/files/47p3z02kwu83


The `response` shows that the File API stored the specified `display_name` for the uploaded file and a `uri` to reference the file in Gemini API calls. Use `response` to track how uploaded files are mapped to URIs.

Depending on your use case, you can also store the URIs in structures such as a `dict` or a database.

#### Verify image file upload and get metadata

You can verify the API successfully stored the uploaded file and get its metadata by calling [`files.get`](https://ai.google.dev/api/rest/v1beta/files/get) through the SDK. Only the `name` (and by extension, the `uri`) are unique. Use `display_name` to identify files only if you manage uniqueness yourself.

In [17]:
file = genai.get_file(name=sample_file.name)
print(f"Retrieved file '{file.display_name}' as: {sample_file.uri}")

Retrieved file 'Jetpack drawing' as: https://generativelanguage.googleapis.com/v1beta/files/47p3z02kwu83


Depending on your use case, you can store the URIs in structures, such as a `dict` or a database.

#### Prompt with the uploaded image and text

After uploading the file, you can make GenerateContent requests that reference the File API URI. Select the generative model and provide it with a text prompt and the uploaded image.

In [18]:
# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Prompt the model with text and the previously uploaded image.
response = model.generate_content([sample_file, "Describe how this product might be manufactured."])

Markdown(response.text)

Manufacturing a jetpack backpack like this would be a complex process involving several stages and specialized expertise:

1. **Design and Engineering:**  CAD software would be used to create detailed blueprints of the backpack's structure, including the placement of the boosters, battery, charging port, and other components.  Computational fluid dynamics (CFD) would be employed to simulate the airflow and optimize booster design for thrust and stability. Material selection (lightweight yet durable fabrics, high-temperature resistant metals for boosters, etc.) is crucial.

2. **Booster Production:**  This would likely involve metal 3D printing or CNC machining for intricate parts, followed by assembly and rigorous testing in controlled environments.  Nozzles and valves would require precise manufacturing tolerances.  The steam generation system (likely miniaturized boiler and heat exchanger) would need to be compact, efficient, and safe.

3. **Battery Production:**  High-density, fast-charging battery cells would be required.  This could involve partnerships with existing battery manufacturers or specialized cell fabrication. Battery management systems (BMS) are critical for safe operation and optimal power delivery.

4. **Electronics and Controls:**  Circuit boards would be designed and manufactured to control the boosters, monitor battery levels, and manage the user interface (potentially a small display or voice activation). Firmware would be developed and tested to ensure reliable and safe operation.

5. **Backpack Assembly:**  The backpack itself would be fabricated using techniques similar to high-performance backpacks.  Reinforced stitching, durable fabrics, and padded straps would be essential for comfort and safety.  Integrating the boosters, battery, and control system into the backpack structure would require precision assembly and quality control.

6. **Testing and Certification:**  Extensive testing would be necessary to ensure safety and performance. This would include flight tests with dummies, environmental testing (temperature extremes, humidity, etc.), and durability testing.  Meeting relevant safety and regulatory standards (e.g., FAA regulations) would be crucial for commercial viability.

7. **Mass Production:**  If successful in testing, the manufacturing process would be scaled up.  This could involve setting up automated production lines for certain components (like the backpack body), while other parts (like the boosters) might require more specialized assembly.

8. **Quality Control:**  Stringent quality control throughout the manufacturing process is paramount.  Regular inspections, performance tests, and rigorous safety checks are crucial to ensure a reliable and safe product.


## Capabilties

This section outlines specific vision capabilities of the Gemini model, including object detection and bounding box coordinates.

### Get bounding boxes

Gemini models are trained to return bounding box coordinates as relative widths or heights in the range of [0, 1]. These values are then scaled by 1000 and converted to integers. Effectively, the coordinates represent the bounding box on a 1000x1000 pixel version of the image. Therefore, you'll need to convert these coordinates back to the dimensions of your original image to accurately map the bounding boxes.

In [20]:
# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Create a prompt to detect bounding boxes.
prompt = "Return a bounding box for each of the objects in this image in [ymin, xmin, ymax, xmax] format."
response = model.generate_content([sample_file, prompt])

Markdown(response.text)

1) padded strap support: [203, 82, 361, 317]
2) fits 18" laptop: [159, 526, 200, 783]
3) lightweight, looks like a normal backpack: [264, 592, 506, 926]
4) usb-c charging: [621, 103, 720, 288]
5) retractable boosters: [610, 644, 694, 884]
6) steam-powered, green/clean: [844, 661, 949, 977]
7) 15-min battery life: [750, 40, 902, 210]
8) jetpack backpack: [70, 352, 132, 725]

The model returns bounding box coordinates in the format
`[ymin, xmin, ymax, xmax]`. To convert these normalized coordinates
to the pixel coordinates of your original image, follow these steps:

1.    Divide each output coordinate by 1000.
1.    Multiply the x-coordinates by the original image width.
1.    Multiply the y-coordinates by the original image height.

To explore more detailed examples of generating bounding box coordinates and
visualizing them on images, review our [Object Detection cookbook example](https://github.com/google-gemini/cookbook/blob/main/examples/Object_detection.ipynb).

## Prompting with video

In this tutorial, you will upload a video using the File API and generate content based on those images.

### Technical details (video)

Gemini 1.5 Pro and Flash support up to approximately an hour of video data.

Video must be in one of the following video format [MIME types](https://developers.google.com/drive/api/guides/ref-export-formats):
  -   `video/mp4`
  -   `video/mpeg`
  -   `video/mov`
  -   `video/avi`
  -   `video/x-flv`
  -   `video/mpg`
  -   `video/webm`
  -   `video/wmv`
  -   `video/3gpp`

The File API service currently extracts image frames from videos at 1 frame per second (FPS) and audio at 1Kbps, single channel, adding timestamps every second. These rates are subject to change in the future for improvements in inference.

**NOTE:** The finer details of fast action sequences may be lost at the 1 FPS frame sampling rate. Consider slowing down high-speed clips for improved inference quality.

Individual frames are 258 tokens, and audio is 32 tokens per second. With metadata, each second of video becomes ~300 tokens, which means a 1M context window can fit slightly less than an hour of video.

To ask questions about time-stamped locations, use the format `MM:SS`, where the first two digits represent minutes and the last two digits represent seconds.

For best results:

*   Use one video per prompt.
*   If using a single video, place the text prompt after the video.

### Upload a video file to the File API

**NOTE**: The File API lets you store up to 20 GB of files per project, with a per-file maximum size of 2 GB. Files are stored for 48 hours. They can be accessed in that period with your API key, but they cannot be downloaded using any API. It is available at no cost in all regions where the Gemini API is available.

The File API accepts video file formats directly. This example uses the short NASA film ["Jupiter's Great Red Spot Shrinks and Grows"](https://www.youtube.com/watch?v=JDi4IdtvDVE0). Credit: Goddard Space Flight Center (GSFC)/David Ladd (2018).

> "Jupiter's Great Red Spot Shrinks and Grows" is in the public domain and does not show identifiable people. ([NASA image and media usage guidelines.](https://www.nasa.gov/nasa-brand-center/images-and-media/))

Start by retrieving the short video:

In [21]:
!wget https://storage.googleapis.com/generativeai-downloads/images/GreatRedSpot.mp4

--2024-12-03 03:14:25--  https://storage.googleapis.com/generativeai-downloads/images/GreatRedSpot.mp4
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.194.207, 142.250.152.207, 173.194.195.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.194.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 238090979 (227M) [video/mp4]
Saving to: ‘GreatRedSpot.mp4’

GreatRedSpot.mp4    100%[===================>] 227.06M   101MB/s    in 2.2s    

2024-12-03 03:14:27 (101 MB/s) - ‘GreatRedSpot.mp4’ saved [238090979/238090979]



Upload the video to the File API and print the URI.

In [22]:
video_file_name = "GreatRedSpot.mp4"

print(f"Uploading file...")
video_file = genai.upload_file(path=video_file_name)
print(f"Completed upload: {video_file.uri}")

Uploading file...
Completed upload: https://generativelanguage.googleapis.com/v1beta/files/bl9ais8hmuiq


### Verify file upload and check state

Verify the API has successfully received the files by calling the [`files.get`](https://ai.google.dev/api/rest/v1beta/files/get) method.

**NOTE**: Video files have a `State` field in the File API. When a video is uploaded, it will be in the `PROCESSING` state until it is ready for inference. Only `ACTIVE` files can be used for model inference.

In [23]:
import time

# Check whether the file is ready to be used.
while video_file.state.name == "PROCESSING":
    print('.', end='')
    time.sleep(10)
    video_file = genai.get_file(video_file.name)

if video_file.state.name == "FAILED":
  raise ValueError(video_file.state.name)

.

### Prompt with a video and text

Once the uploaded video is in the `ACTIVE` state, you can make `GenerateContent` requests that specify the File API URI for that video. Select the generative model and provide it with the uploaded video and a text prompt.

In [24]:
# Create the prompt.
prompt = "Summarize this video. Then create a quiz with answer key based on the information in the video."

# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Make the LLM request.
print("Making LLM inference request...")
response = model.generate_content([video_file, prompt],
                                  request_options={"timeout": 600})

# Print the response, rendering any Markdown
Markdown(response.text)

Making LLM inference request...


Sure, here's a summary of the video and a quiz with answers:

Jupiter is the largest and oldest planet in our solar system. It's considered a gas giant and is made of the same elements as stars, but it wasn't massive enough to ignite. It has a turbulent, swirling interior of gasses and liquids that creates stripes and swirling storms. One of the most iconic and studied storms on Jupiter is the Great Red Spot, which is an anticyclone that has been raging for more than a century. Since there's no land mass on Jupiter, the storm hasn't been slowed down. However, data from NASA suggests that the spot is getting smaller and has been since 1995, but it's also growing taller. NASA scientists are using data collected from the Voyager, Hubble, and Juno missions to further investigate the Great Red Spot, hoping to unlock more of the planet's mysteries.

**Quiz Questions**

1. Which planet is the biggest and oldest in our solar system?
2. What type of storm is the Great Red Spot?
3. How long has the Great Red Spot been raging?
4. What's happening to the size and shape of the Great Red Spot?
5. What three NASA missions have helped scientists collect data to study the Great Red Spot?

**Answer Key**

1. Jupiter
2. Anticyclone
3. More than a century
4. It's shrinking, becoming more round, and growing taller.
5. Voyager, Hubble, and Juno

### Refer to timestamps in the content

You can use timestamps of the form `MM:SS` to refer to specific moments in the video.

In [25]:
# Create the prompt.
prompt = "What are the examples given at 01:05 and 01:19 supposed to show us?"

# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Make the LLM request.
print("Making LLM inference request...")
response = model.generate_content([prompt, video_file],
                                  request_options={"timeout": 600})
Markdown(response.text)

Making LLM inference request...


At 01:05, an ice skater shows us how pulling in her arms increases her rotational speed. At 01:19, a potter shows us that as they draw the clay upwards, the pot takes on a taller form but gets smaller in diameter. These examples are used to show that as the Great Red Spot gets smaller, it would be expected to get faster, but it actually gets taller and rounder.

### Transcribe video and provide visual descriptions

The Gemini models can transcribe and provide visual descriptions of video content
by processing both the audio track and visual frames.
For visual descriptions, the model samples the video at a rate of **1 frame
per second**. This sampling rate may affect the level of detail in the
descriptions, particularly for videos with rapidly changing visuals.

In [27]:
# Create the prompt.
prompt = "Transcribe the audio from this video, giving timestamps for salient events in the video. Also provide visual descriptions."

# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Make the LLM request.
print("Making LLM inference request...")
response = model.generate_content([video_file, prompt],
                                  request_options={"timeout": 600})
Markdown(response.text)

Making LLM inference request...


Sure, here is the transcription and timestamps for the video about the Great Red Spot on Jupiter:

[00:00:00 to 00:00:16] Jupiter appears out of the darkness against the backdrop of the Milky Way. As the video zooms in, details of Jupiter’s cloud patterns begin to emerge.
[00:00:16 to 00:00:21] Jupiter rotates as the Milky Way moves past it.
[00:00:21 to 00:00:26] Jupiter takes its full shape against a black background.
[00:00:26 to 00:00:32] The Great Red Spot is visible. The video zooms in on the spot.
[00:00:32 to 00:00:33] Jupiter with the Great Red Spot is centered, the Milky Way is again in the background.
[00:00:33 to 00:00:42] Jupiter rotates and expands as the video moves in to the Great Red Spot, the Milky Way passes by again.
[00:00:42 to 00:00:47] Jupiter is seen in its entirety. The Great Red Spot is not easily visible.
[00:00:47 to 00:01:00] Jupiter rotates against the black background. An inset image of the Great Red Spot appears showing its shape in 1995, 2009, and 2015, with a dotted line tracing its edge. The spot has become smaller over time.
[00:01:00 to 00:01:05] The video zooms in to the Great Red Spot.
[00:01:05 to 00:01:13] A figure skater is spinning on an ice rink.
[00:01:13 to 00:01:17] The video zooms in on the Great Red Spot from a low angle.
[00:01:17 to 00:01:25] A black-and-white still image and then a clip of a potter shaping clay on a potter's wheel are shown, side-by-side. There is also a graph showing the growth of the Great Red Spot in recent years.
[00:01:25 to 00:01:30] Jupiter against a black background. The inset image reappears, showing its shrinking over time.
[00:01:30 to 00:01:36] The Great Red Spot is now the size of Earth. The Milky Way moves past. 
[00:01:36 to 00:01:46] Two spacecraft and Jupiter are seen in space.
[00:01:46 to 00:01:54] Jupiter against the Milky Way background. The Great Red Spot is seen again, with the video zooming in closer and closer.
[00:01:54 to 00:01:57] The video stays in place over the Great Red Spot as the clouds move.
[00:01:58] The video returns to black.
[00:01:59 to 00:02:06] A NASA Goddard Space Flight Center logo appears on the screen as the Earth is visible in the background.

## List files

You can list all uploaded files and their URIs using `files.list_files()`.

In [28]:
# List all files
for file in genai.list_files():
    print(f"{file.display_name}, URI: {file.uri}")

GreatRedSpot.mp4, URI: https://generativelanguage.googleapis.com/v1beta/files/bl9ais8hmuiq
Jetpack drawing, URI: https://generativelanguage.googleapis.com/v1beta/files/47p3z02kwu83


## Delete files

Files are automatically deleted after 2 days. You can also manually delete them using `files.delete()`.

In [29]:
genai.delete_file(video_file.name)
print(f'Deleted file {video_file.uri}')

Deleted file https://generativelanguage.googleapis.com/v1beta/files/bl9ais8hmuiq


# Assignment To Identify License Plate

In [30]:
# Upload the file and print a confirmation.
license_plate = genai.upload_file(path="mobil_putih.JPG",
                            display_name="License Plate")

print(f"Uploaded file '{license_plate.display_name}' as: {license_plate.uri}")

Uploaded file 'License Plate' as: https://generativelanguage.googleapis.com/v1beta/files/jbyxchoo305m


In [32]:
# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Prompt the model with text and the previously uploaded image.
response = model.generate_content([license_plate, "read the plate number of the image with output example (‘plat_no’: ‘B 1234 ABC’,	‘vehicle’: ‘car’,‘vehicle_type’: ‘sedan’,	‘color’: ‘red’,	‘gate_open’: ‘2024-12-02 18.15.01’,	‘gate_closed’: ‘N/A’,)"])

Markdown(response.text)

```json
{
‘plat_no’: ‘BK 1059 ADB’,
‘vehicle’: ‘car’,
‘vehicle_type’: ‘SUV’,
‘color’: ‘white’,
‘gate_open’: ‘N/A’,
‘gate_closed’: ‘N/A’
}
```